# Student ID, name of each team member
107021115 錢珈鋒
110065536 陳俊元
110062645 王昱文

# How did you preprocess data, e.g. cleaning, feature engineering, etc?
First we use the idea TA share on the notebook but the result not even touch the baseline,there fore we start to extract as mush features as we can,such as title,topic,channel,adj in the content,the length of the content,date,title,author's name,after that we try all the combination we can possibly image.Finally, we find out the combination of the number of the positive an author get ,date and topic are three of the best combination.

# How did you build the classifier, e.g. model, training algorithm, special techniques, etc?

Other than the feature, we have googled the sentiment analysis techniques,based on the study we have applied LDA,LLE and all sorts of the distribution and it rather than helping us getting the higger score than lower the socre which was a crush on us.Because all of the research told us that LDA LLE distribution will definitely help us a lot which was a oppositie of the outcome.We still wanna know how to get better score through LDA.

# Conclusions, including interesting findings, pitfalls, takeaway lessons, etc.

We have learned a lot by using beautifulsoup,we now have the knowledge of extracting the feature we are want.At the first time we upload our prediction it seems great on validation data but terrible on testing data.There's still a big progress we need to try because some of the classmate can reach 0.6 and even better without using DL skills, really looking forward to learn how they did it.

In [1]:
import re
import nltk
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier, SGDRegressor, LogisticRegression, LinearRegression
from sklearn.metrics import roc_auc_score, mean_squared_error, accuracy_score
from sklearn import neighbors
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.manifold import LocallyLinearEmbedding
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/yuan65536/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/yuan65536/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/yuan65536/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


# Read Data

In [2]:
df = pd.read_csv('./data/train.csv')
df.head(5)

df_test = pd.read_csv('./data/test.csv')

In [3]:
#use bf4 to convert html
def preprocessor(text):
    # remove HTML tags
    text = BeautifulSoup(text, 'html.parser').get_text()
    
    # regex for matching emoticons, keep emoticons, ex: :), :-P, :-D
    r = '(?::|;|=|X)(?:-)?(?:\)|\(|D|P)'
    emoticons = re.findall(r, text)
    text = re.sub(r, '', text)
    
    # convert to lowercase and append all emoticons behind (with space in between)
    # replace('-','') removes nose of emoticons
    text = re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-','')
    return text

In [4]:
#tokenizer
def tokenizer_stem_nostop(text):
    porter = PorterStemmer()
    return [porter.stem(w) for w in re.split('\s+', text.strip()) \
            if w not in stop and re.match('[a-zA-Z]+', w)]

In [5]:
x = df['Page content'].values
y = df['Popularity'].values
print('training data points: ', x.shape[0])

x_test = df_test['Page content'].values
ID_test = df_test['Id'].values

training data points:  27643


x_train, x_valid, y_train, y_valid = train_test_split(
    x, y, test_size = 0.2, random_state = 0
)

# Features extract
content length

In [6]:
def countbodynumber(text):
    count = 0
    for i in text.split():
        count += 1
    
    return count

In [7]:
def get_date(text):
    soup = BeautifulSoup(text, 'html.parser')
    tag = soup.find('time')
    ch = str(tag)
    ch = ch.split(' ')
    ch = ch[1].split('"')
    ch = ch[1].split(',')
    return ch[0]



In [8]:
def get_channel(text):
    soup = BeautifulSoup(text, 'html.parser')
    tag = soup.find('article')
    ch = str(tag)
    ch = ch.split('>')
    ch = ch[0].split(' ')
    ch = ch[1].split('"')
    return (ch[1])
  

extract title

In [9]:
def gettitle(text):
    soup = BeautifulSoup(text, 'html.parser')
    title_tag = soup.find(class_ = 'title')
    
    return title_tag.string

extract author

In [10]:
def getAuthorName(text):
    text = text.replace("</a>", " ")
    text = text.replace("by", "")
    text = text.replace("By", "")
    # remove HTML tags
    text = BeautifulSoup(text, 'html.parser').get_text()
    
    # regex for matching emoticons, keep emoticons, ex: :), :-P, :-D
    r = '(?::|;|=|X)(?:-)?(?:\)|\(|D|P)'
    emoticons = re.findall(r, text)
    text = re.sub(r, '', text)
    text = re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-','')
    count = 0
    l = []
    for i in range(len(text)):
        if text[i] == " ":
            if count == 2:
                break
            if count == 1:
                l.append(" ") 
                count = 2
            
        if text[i] != " ":
            if count == 0:
                count = 1
            l.append(text[i])

    s = ''.join(l)
    return s

def getAuthorList(X):
    Name = []
    for i in range(len(X)):
        name = getAuthorName(X[i])
        Name.append(name)
    return Name

get part of speech

In [11]:
def get_part_of_speech(title):
    tokens = nltk.word_tokenize(title)
    pos_tags = nltk.pos_tag(tokens)
    
    temp = []
    for word, pos in pos_tags:
        if (pos == 'JJ' or pos == 'JJR' or pos == 'JJS'):
            temp.append(word)
    
    pos_str = ' '.join(temp)
    
    return pos_str

In [12]:
def getAuthorCount(AuthorCountLabelList,NameList):
    Name=AuthorCountLabelList[:,0].tolist()
    NameCount=AuthorCountLabelList[:,1].tolist()
    AuthorCountList=[]
    for i in range(len(NameList)):
        if NameList[i]  in Name:
            index=Name.index(NameList[i])
            AuthorCountList.append(float(NameCount[index]))
        else:
            AuthorCountList.append(0)
    return AuthorCountList


get date

In [13]:
'''def get_date(text):
    soup = BeautifulSoup(text, 'html.parser')
    date_tag = soup.find('time')
    
    return date_tag.text

'''

"def get_date(text):\n    soup = BeautifulSoup(text, 'html.parser')\n    date_tag = soup.find('time')\n    \n    return date_tag.text\n\n"

In [15]:
def get_topic(text):
    soup = BeautifulSoup(text, 'html.parser')
    topics = [str(i.string) for i in soup.find_all('a') if i.parent.name == 'footer']
    topics = " ".join(topics)

    return topics

def getTopicsList(X):
    Topics=[]
    for i in range(len(X)):
        topics=getTopics(X[i])
        Topics.append(topics)
    return Topics

In [16]:
def get_features(data):
    title_list = []
    date_list = []
    topic_list = []
    pos = []
    content_length = []
    author_list = getAuthorList(data)
    channel=[]
    for i in range(len(data)):
        title_tmp = gettitle(data[i])
        title_list.append(title_tmp)
        
        word = get_part_of_speech(title_tmp)
        pos.append(word)
        
        
        ch = get_channel(data[i])
        channel.append(ch)
        
        
        text = preprocessor(data[i])
        content_length.append(countbodynumber(text))
        
        date_tmp = get_date(str(data[i]))
        if(date_tmp=='='):date_list.append('Mon')
        else:date_list.append(date_tmp)
        
        topic_list.append(get_topic(data[i]))
        
        
    return title_list, author_list, pos, content_length, date_list, topic_list,channel

In [17]:
def buildAuthorCountLabel(NameList):
    Name=[]
    NameCount=[]
    for j in range(len(NameList)):
        if NameList[j] not in Name:
            Name.append(NameList[j])
            NameCount.append(1)
        else :
            index=Name.index(NameList[j])
            temp=NameCount[index]
            NameCount[index]=temp+1
    Name_np = np.array(Name)
    NameCount_np = np.array(NameCount)
    AuthorCountLebalList = np.concatenate((Name_np.reshape(-1,1),NameCount_np.reshape(-1,1)), axis=1)
    return AuthorCountLebalList

def getAuthorCount(AuthorCountLabelList,NameList):
    Name=AuthorCountLabelList[:,0].tolist()
    NameCount=AuthorCountLabelList[:,1].tolist()
    AuthorCountList=[]
    for i in range(len(NameList)):
        if NameList[i]  in Name:
            index=Name.index(NameList[i])
            AuthorCountList.append(float(NameCount[index]))
        else:
            AuthorCountList.append(0)
    return AuthorCountList

# Preprocessing

In [18]:
'''
title_train, authorf_train, posf_train, content_lengthf_train, datef_train, topic_train,channel_train = get_features(x_train)
title_valid, authorf_valid, posf_valid, content_lengthf_valid, datef_valid, topic_valid,channel_valid = get_features(x_valid)


NameList_1 = []
NameList_2 = []

AuthorCountLabelList_t=buildAuthorCountLabel(NameList_1)
AuthorCountList_t=getAuthorCount(AuthorCountLabelList_t,authorf_train)

AuthorCountLabelList_v=buildAuthorCountLabel(NameList_2)
AuthorCountList_v=getAuthorCount(AuthorCountLabelList_v,authorf_valid)



tt = np.array(title_train)
at = np.array(authorf_train)
pt = np.array(posf_train)
dt = np.array(datef_train)
to_t = np.array(topic_train)
ct = np.array(channel_train)
ac_t = np.array(AuthorCountList_t)

tv = np.array(title_valid)
av = np.array(authorf_valid)
pv = np.array(posf_valid)
dv = np.array(datef_valid)
to_v = np.array(topic_valid)
cv = np.array(channel_valid)
ac_v = np.array(AuthorCountList_v)
'''

'\ntitle_train, authorf_train, posf_train, content_lengthf_train, datef_train, topic_train,channel_train = get_features(x_train)\ntitle_valid, authorf_valid, posf_valid, content_lengthf_valid, datef_valid, topic_valid,channel_valid = get_features(x_valid)\n\n\nNameList_1 = []\nNameList_2 = []\n\nAuthorCountLabelList_t=buildAuthorCountLabel(NameList_1)\nAuthorCountList_t=getAuthorCount(AuthorCountLabelList_t,authorf_train)\n\nAuthorCountLabelList_v=buildAuthorCountLabel(NameList_2)\nAuthorCountList_v=getAuthorCount(AuthorCountLabelList_v,authorf_valid)\n\n\n\ntt = np.array(title_train)\nat = np.array(authorf_train)\npt = np.array(posf_train)\ndt = np.array(datef_train)\nto_t = np.array(topic_train)\nct = np.array(channel_train)\nac_t = np.array(AuthorCountList_t)\n\ntv = np.array(title_valid)\nav = np.array(authorf_valid)\npv = np.array(posf_valid)\ndv = np.array(datef_valid)\nto_v = np.array(topic_valid)\ncv = np.array(channel_valid)\nac_v = np.array(AuthorCountList_v)\n'

In [19]:
title_train, authorf_train, posf_train, content_lengthf_train, datef_train, topic_train,channel_train = get_features(x)

In [20]:
NameList=getAuthorList(x)
NameList_1 = []
j=0
for i in range(len(NameList)):
    if y[i]==1:
        NameList_1.append(NameList[i])
        j=j+1
    else:
        NameList_1.append(NameList[i])
        
AuthorCountLabelList=buildAuthorCountLabel(NameList_1)
AuthorCountList=getAuthorCount(AuthorCountLabelList,authorf_train)
AuthorCountList = np.array(AuthorCountList)

In [21]:
datef_train=np.array(datef_train)
topic_train=np.array(topic_train)

hashvec = HashingVectorizer(n_features=2**13,
                            preprocessor=preprocessor,
                            tokenizer=tokenizer_stem_nostop)

onehot_date = OneHotEncoder(handle_unknown = 'ignore')


Date_onehot_train = onehot_date.fit_transform(datef_train.reshape(-1, 1))
Topic_hash_train = hashvec.transform(topic_train)


X = np.concatenate((Date_onehot_train.toarray(), Topic_hash_train.toarray(),AuthorCountList.reshape(-1,1)), axis=1)


In [22]:
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)
NameList = np.array(NameList)

print(len(NameList))

27643


clt = np.array(content_lengthf_train)
clv = np.array(content_lengthf_valid)

In [23]:
def bow(tag_t, tag_v):
    count = CountVectorizer(ngram_range = (1, 1),
                            preprocessor = preprocessor,
                            tokenizer = tokenizer_stem_nostop)
    
    res_t = count.fit_transform(tag_t)
    res_v = count.transform(tag_v)
    
    return res_t, res_v

In [24]:
def hash_fun(tag_t, tag_v, fea):
    hash_f = HashingVectorizer(n_features = fea,
                                   preprocessor = preprocessor,
                                   tokenizer = tokenizer_stem_nostop)
    
    res_t = hash_f.fit_transform(tag_t)
    res_v = hash_f.transform(tag_v)
    
    return res_t, res_v

In [25]:
def ohe(tag_t, tag_v):
    onehot = OneHotEncoder(handle_unknown = 'ignore')
    
    res_t = onehot.fit_transform(tag_t.reshape(-1, 1))
    res_v = onehot._transform(tag_v.reshape(-1, 1))
    
    return res_t, res_v

In [26]:
def tfidf_fun(tag_t, tag_v):
    tfidf = TfidfVectorizer(ngram_range = (1,1),
                            preprocessor = preprocessor,
                            tokenizer = tokenizer_stem_nostop)
    
    res_t = tfidf.fit_transform(tag_t)
    res_v = tfidf.transform(tag_v)
    
    return res_t, res_v

In [27]:
def lda_fun(t, v, n):
    lda = LatentDirichletAllocation(n_components = n, random_state = 42)
    
    res_t = lda.fit_transform(t)
    res_v = lda.transform(v)
    
    return res_t, res_v

In [28]:
'''#t, t2 = bow(tt, tv)
t,t2  = hash_fun(tt,tv,2**10)   #title
a, a2 = hash_fun(at, av, 2**11) #author
p, p2 = bow(pt, pv)             #pos
d, d2 = ohe(dt, dv)     #date


to, to2 = hash_fun(to_t, to_v, 2**10) #topic

ct_t, cv_v = hash_fun(ct, cv, 2**10) #channel'''

'#t, t2 = bow(tt, tv)\nt,t2  = hash_fun(tt,tv,2**10)   #title\na, a2 = hash_fun(at, av, 2**11) #author\np, p2 = bow(pt, pv)             #pos\nd, d2 = ohe(dt, dv)     #date\n\n\nto, to2 = hash_fun(to_t, to_v, 2**10) #topic\n\nct_t, cv_v = hash_fun(ct, cv, 2**10) #channel'

In [29]:
'''title_t = t.toarray()
author_t = a.toarray()
pos_t = p.toarray()
date_t = d.toarray()
topic_t = to.toarray()
ct_t = ct_t.toarray()
Acount_t=ac_t.reshape(-1,1)
#doc_train_temp = np.array(doc_train)
#train_tfidf = doc_train_temp.toarray()



title_v = t2.toarray()
author_v = a2.toarray()
pos_v = p2.toarray()
date_v = d2.toarray()
topic_v = to2.toarray()
cv_v = cv_v.toarray()
Acount_v=ac_v.reshape(-1,1)

#doc_valid_temp = np.array(doc_valid)
#valid_tfidf = doc_valid_temp.toarray()
'''

'title_t = t.toarray()\nauthor_t = a.toarray()\npos_t = p.toarray()\ndate_t = d.toarray()\ntopic_t = to.toarray()\nct_t = ct_t.toarray()\nAcount_t=ac_t.reshape(-1,1)\n#doc_train_temp = np.array(doc_train)\n#train_tfidf = doc_train_temp.toarray()\n\n\n\ntitle_v = t2.toarray()\nauthor_v = a2.toarray()\npos_v = p2.toarray()\ndate_v = d2.toarray()\ntopic_v = to2.toarray()\ncv_v = cv_v.toarray()\nAcount_v=ac_v.reshape(-1,1)\n\n#doc_valid_temp = np.array(doc_valid)\n#valid_tfidf = doc_valid_temp.toarray()\n'

In [30]:
'''x_t_feature = np.concatenate((date_t,topic_t,Acount_t), axis = 1)
x_v_feature = np.concatenate((date_v,topic_v,Acount_v), axis = 1)'''

'x_t_feature = np.concatenate((date_t,topic_t,Acount_t), axis = 1)\nx_v_feature = np.concatenate((date_v,topic_v,Acount_v), axis = 1)'

In [31]:
x_train, x_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.3, random_state = 0)


lr = LogisticRegression( solver = "liblinear")
lr.fit(X, y)

y_train_pred = lr.predict(x_train)
y_valid_pred = lr.predict(x_valid)


print('train roc: %f, valid roc: %f' % (roc_auc_score(y_train, y_train_pred),
                                        roc_auc_score(y_valid, y_valid_pred)))
    
print('train acc: %f, valid roc: %f' % (accuracy_score(y_train, y_train_pred),
                                        accuracy_score(y_valid, y_valid_pred)))

train roc: 0.624164, valid roc: 0.625737
train acc: 0.624651, valid roc: 0.626553


# Testing

In [32]:
title_test, authorf_test, posf_test, content_lengthf_test, datef_test, topic_test,channel_test = get_features(x_test)

In [33]:
NameList_test=getAuthorList(x_test)
AuthorCountList_t=getAuthorCount(AuthorCountLabelList,NameList_test)


datef_test=np.array(datef_test)
topic_test=np.array(topic_test)



AuthorCountList_test = np.array(AuthorCountList_t)
Date_onehot_test = onehot_date.transform(datef_test.reshape(-1, 1))
Topic_hash_test  = hashvec.transform(topic_test)


test = np.concatenate((Date_onehot_test.toarray(), Topic_hash_test.toarray(),AuthorCountList_test.reshape(-1,1)), axis=1)


In [ ]:
#print(topic_test)
#print(e_test.toarray())
#print(AuthorCountList_test)

In [ ]:
#import sys
#import numpy
#numpy.set_printoptions(threshold=sys.maxsize)

#print(AuthorCountList_test)

ttt = np.array(title_test)
att = np.array(authorf_test)
ptt = np.array(posf_test)
dtt = np.array(datef_test)
to_tt = np.array(topic_test)
cltt = np.array(content_lengthf_test)

t, t2 = bow(tt, ttt)
a, a2 = hash_fun(at, att, 2**11) #2**11
p, p2 = bow(pt, ptt)
content_length_tt = cltt.reshape(len(x_test), 1)
d, d2 = bow(dt, dtt)
to, to2 = hash_fun(to_t, to_tt, 2**10) #2**10

title_tt = t2.toarray()
author_tt = a2.toarray()
pos_tt = p2.toarray()
date_tt = d2.toarray()
topic_tt = to2.toarray()

x_tt_feature = np.concatenate((author_tt, content_length_tt, date_tt, topic_tt), axis = 1)

# Write csv

In [44]:
result = lr.predict(test)
d = {'Id': ID_test,'Popularity': result}
output_test_pred = pd.DataFrame(data = d)
output_test_pred.to_csv('y_predict.csv', index = False)

In [51]:
print(output_test_pred)

          Id  Popularity
0      27643          -1
1      27644           1
2      27645          -1
3      27646          -1
4      27647          -1
...      ...         ...
11842  39485           1
11843  39486          -1
11844  39487           1
11845  39488           1
11846  39489          -1

[11847 rows x 2 columns]
